In [162]:
from tools import *
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GATConv
from tqdm import tqdm  


In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, num_features,output_layer, hidden_dim=128,hidden_dim_2=128,hidden_dim_3=128):
        super().__init__()
        self.conv1 = SAGEConv(num_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim_2)
        self.conv3 = SAGEConv(hidden_dim_2, hidden_dim_3)
        self.conv4 = SAGEConv(hidden_dim_2, hidden_dim_3)
        self.conv5 = SAGEConv(hidden_dim_2, hidden_dim_3)

         
        self.lin = torch.nn.Linear(hidden_dim_3, output_layer)

    def forward(self, data,dropout=0.22):
        x, edge_index = data.x[:,:4], data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.dropout(x, p=dropout, train=self.training)
        x = torch.relu(self.conv2(x, edge_index))
        x = torch.dropout(x, p=dropout, train=self.training)
        x = torch.relu(self.conv3(x, edge_index))
        x = torch.dropout(x, p=dropout, train=self.training)
        x = torch.relu(self.conv4(x, edge_index))
        x = torch.dropout(x, p=dropout, train=self.training)
        x = torch.relu(self.conv5(x, edge_index))
        # x = torch.dropout(x, p=dropout, train=self.training)

        x = self.lin(x)
        return x



In [164]:
def evaluate(model, graphs,N):
    model.eval()
    total_correct = 0
    total_samples = 0
    for data in graphs:
            
        with torch.no_grad():
            i=0
        
            data = data.to(torch.device('cpu'))
            # print("prediction: ",i)
            out = model(data)
            i+=1
            pred = out.argmax(dim=1)
        
            num=int(data.x[0,14])
            suffixe=int(data.x[0,15])
            if suffixe!=-1:
                graphe_name='Graphe('+str(num)+')_'+str(suffixe)+'_features.json'
            else:
                graphe_name='Graphe('+str(num)+')_features.json'
            path='graphes_JSON_Complet/'+str(N)+'/test'
            print(graphe_name)
            print(path)


            add_prediction_to_json(pred,graphe_name,path)
            # print("actual: ",data.y)
            # print("pred: ",pred)
            total_correct += (pred == data.y).sum().item()
            total_samples += data.y.size(0)

    accuracy = total_correct / total_samples
    return accuracy

In [165]:
def train(model,epochs,train_loader,optimizer):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        for data in train_loader:
            data = data.to(torch.device('cpu'))
            optimizer.zero_grad()
            out = model(data)
            targets = data.y.long()
            loss = F.cross_entropy(out, targets)
            loss.backward()
            optimizer.step()
            _, pred = out.max(dim=-1)
            correct = (pred == targets).sum().item()
            total_loss += loss.item()
            total_correct += correct
            total_samples += targets.size(0)

        if (epoch % 10 == 0) or (epoch == 99):
            epoch_loss = total_loss / len(train_loader)
            epoch_acc = total_correct / total_samples
            print(f'Epoch {epoch+1:3d} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')


In [ ]:
num_features=4
epochs=250
N=100

In [ ]:
for i in range(2,N):
    # loading graphs
    train_graphs, test_graphs = load_graphs_by_ncpu(i)
    train_loader = DataLoader(train_graphs, batch_size=4, shuffle=True)
    

    model = GraphSAGE(num_features,i).to(torch.device('cpu'))  
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # training 
    print("********Training on clusters with ",i," machines**********")
    train(model,epochs,train_loader,optimizer)
    # testing
    acc  = evaluate(model,test_graphs,i)
    print(f'Test Accuracy: {acc:.4f}')

Loaded 44 training and 12 testing graphs for n_cpus=2
********Training on clusters with  2  machines**********
Epoch   1 | Loss: 0.6969 | Acc: 0.5511
Epoch  11 | Loss: 0.6113 | Acc: 0.5875
Epoch  21 | Loss: 0.5737 | Acc: 0.6049
Epoch  31 | Loss: 0.4909 | Acc: 0.6794
Epoch  41 | Loss: 0.3453 | Acc: 0.7348
Epoch  51 | Loss: 0.3606 | Acc: 0.7158
Epoch  61 | Loss: 0.2471 | Acc: 0.7972
Epoch  71 | Loss: 0.1819 | Acc: 0.8492
Epoch  81 | Loss: 0.2219 | Acc: 0.8579
Epoch  91 | Loss: 0.2270 | Acc: 0.8510
Epoch 100 | Loss: 0.1734 | Acc: 0.9012
Epoch 101 | Loss: 0.1681 | Acc: 0.8995
Epoch 111 | Loss: 0.1328 | Acc: 0.9151
Epoch 121 | Loss: 0.1423 | Acc: 0.9289
Epoch 131 | Loss: 0.1617 | Acc: 0.8873
Epoch 141 | Loss: 0.1425 | Acc: 0.9272
Epoch 151 | Loss: 0.1239 | Acc: 0.9411
Epoch 161 | Loss: 0.1255 | Acc: 0.9428
Epoch 171 | Loss: 0.1177 | Acc: 0.9341
Epoch 181 | Loss: 0.1276 | Acc: 0.9445
Epoch 191 | Loss: 0.1096 | Acc: 0.9515
Graphe(444)_features.json
graphes_JSON_Complet/2/test
Graphe(4471)_fea